# Assignment 2  - Decision Tree

# Decision Tree Classification of Breast Cancer Patient Data

## Objective
The goal of this project is to develop a decision tree model to classify breast cancer patients' survival status as either **"Alive"** or **"Dead"**. The tasks are:

1. Construct a decision tree using the **Gini impurity** criterion.
2. Fit the model to the dataset and evaluate its performance by plotting validation loss and accuracy, varying the number of epochs (iterations) from 1 to 20.

## Dataset
This dataset was sourced from the SEER Program's November 2017 update, which provides population-based cancer statistics. It includes data on female patients diagnosed with **infiltrating duct and lobular carcinoma** of the breast (histology codes 8522/3) between **2006 and 2010**.

### Input Variables
The dataset contains the following features:

1. **Age**: Patient's age.
2. **Race**: Patient's race (e.g., American Indian/AK Native, Asian/Pacific Islander).
3. **Marital Status**: Marital status of the patient.
4. **T Stage**: AJCC 6th T stage classification for the tumor.
5. **N Stage**: AJCC 6th N stage classification for regional lymph nodes.
6. **6th Stage**: Breast cancer AJCC 6th stage classification.
7. **Differentiate**: Degree of differentiation of the tumor.
8. **Grade**: Tumor grade, indicating its aggressiveness.
9. **A Stage**:
   - **Regional**: Neoplasm that has extended.
   - **Distant**: Neoplasm that has metastasized to distant body parts.
10. **Tumor Size**: Tumor size in millimeters.
11. **Estrogen Status**: Status of estrogen receptors in the tumor.
12. **Progesterone Status**: Status of progesterone receptors in the tumor.
13. **Regional Node Examined**: Number of regional lymph nodes examined.
14. **Regional Node Positive**: Number of regional lymph nodes positive for cancer.
15. **Survival Months**: Number of months the patient survived after diagnosis.

### Output Variable
- **Status**: Survival status of the patient (`Alive` or `Dead`).

## Methodology

1. **Decision Tree Construction**: We will use the **Gini impurity** method for splitting nodes.
2. **Model Training and Evaluation**:
   - The decision tree will be trained on the dataset.
   - We will monitor **validation loss** and **accuracy** while varying the number of epochs from **1 to 20**.
   - Plots of accuracy and validation loss over epochs will be generated to assess the model’s performance.


In [ ]:
# Install pandas, numpy, seaborn, and matplotlib
! pip install pandas numpy seaborn matplotlib

# Install scikit-learn
! pip install scikit-learn

# Install IPython
! pip install ipython

# Install pygraphviz
! pip install pygraphviz

In [1]:
# import all the necessary libraries here
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,precision_score,recall_score
from graphviz import Digraph
from IPython.display import Image, display
import pygraphviz as pgv


Please see that in Breast_Cancer.csv file the Attribute 'T Stage' has a space at the end, remove that for no errors.

In [2]:
df = pd.read_csv('Breast_Cancer.csv') # Replace with noise/noiseless dataset
string_columns = ['Race', 'Marital Status', 'T Stage', 'N Stage', '6th Stage', 'differentiate', 'Grade', 'A Stage', 'Estrogen Status', 'Progesterone Status', 'Status']

for column in string_columns:
    unique_values = df[column].unique()
    value_to_int = {value: idx for idx, value in enumerate(unique_values)}
    df[column] = df[column].map(value_to_int)
df.head()

,Age,Race,Marital Status,T Stage,N Stage,6th Stage,differentiate,Grade,A Stage,Tumor Size,Estrogen Status,Progesterone Status,Regional Node Examined,Reginol Node Positive,Survival Months,Status
0,68,0,0,0,0,0,0,0,0,4,0,0,24,1,60,0
1,50,0,0,1,1,1,1,1,0,35,0,0,14,5,62,0
2,58,0,1,2,2,2,1,1,0,63,0,0,14,7,75,0
3,58,0,0,0,0,0,0,0,0,18,0,0,2,1,84,0
4,47,0,0,1,0,3,0,0,0,41,0,0,3,1,50,0


**Train, Validation, Test split**

In [3]:
train_df,test_df = train_test_split(df,test_size=0.2,random_state=1)
train_df,val_df = train_test_split(train_df,test_size=0.2,random_state=1)
train_df.shape,test_df.shape,val_df.shape

((2575, 16), (805, 16), (644, 16))

In [4]:
df.head()

,Age,Race,Marital Status,T Stage,N Stage,6th Stage,differentiate,Grade,A Stage,Tumor Size,Estrogen Status,Progesterone Status,Regional Node Examined,Reginol Node Positive,Survival Months,Status
0,68,0,0,0,0,0,0,0,0,4,0,0,24,1,60,0
1,50,0,0,1,1,1,1,1,0,35,0,0,14,5,62,0
2,58,0,1,2,2,2,1,1,0,63,0,0,14,7,75,0
3,58,0,0,0,0,0,0,0,0,18,0,0,2,1,84,0
4,47,0,0,1,0,3,0,0,0,41,0,0,3,1,50,0


### Model Implementation

# How the algorithm works

**We'll start with all examples at the root node then:**

**We'll calculate Gini Impurity for splitting on all possible features and pick the one with the Lowest value**

**Then we'll split the data according to the selected feature**

**We'll repeat this  process until stopping criteria is met**

## Key Points:

### Gini Impurity
**Gini Impurity is a measure of impurity or disorder. It is calculated using the following formula:**

$$G = 1 - \sum_{i=1}^{n} p_i^2$$

**Where \( p_i \) is the probability of an element being classified for a particular class.**


### Gini Impurity of Node Split

**The Gini Impurity for a node split is calculated using the following formula:**

$$G_{\text{split}} = \frac{N_{\text{left}}}{N} G_{\text{left}} + \frac{N_{\text{right}}}{N} G_{\text{right}}$$

**Where:**

- \( G_{\text{split}} \) is the Gini Impurity of the split.
- \( N_{\text{left}} \) is the number of samples in the left child node.
- \( N_{\text{right}} \) is the number of samples in the right child node.
- \( N \) is the total number of samples (i.e., \( N = N_{\text{left}} + N_{\text{right}} \)).
- \( G_{\text{left}} \) is the Gini Impurity of the left child node.
- \( G_{\text{right}} \) is the Gini Impurity of the right child node.




### Decision Tree visualization using graphviz

In [5]:
graph = pgv.AGraph(strict=True, directed=True)
graph2 = pgv.AGraph(strict=True, directed=True)

## Start the Implementation

In [6]:

class Node:
    def __init__(self, data):
        self.data = data
        self.children = {}  

class DecisionTree:
    def __init__(self):
        self.root = Node(None)
        self.height = -1

    def find_splits(self, data, column_index):
        """
        Identifies potential split points for a given feature based on Gini Impurity.

        Parameters:
            data (DataFrame): The dataset being used to identify splits.
            column_index (int): The index of the feature to evaluate.

        Returns:
            list: A list of potential split points for the specified feature.
        """
        unique_values = np.unique(data.iloc[:, column_index].values)  # Use iloc for indexing
        return unique_values.tolist()  

    def calculate_gini_impurity(self, data):
        """
        Calculates the Gini Impurity of the dataset.
        
        Parameters:
            data (DataFrame): The dataset for which to calculate Gini Impurity.
        
        Returns:
            float: The Gini Impurity value of the dataset.
        """
        y = data.iloc[:, -1].values  # Accessing last column correctly
        _, counts = np.unique(y, return_counts=True)
        probabilities = counts / counts.sum()
        gini_impurity = 1 - np.sum(probabilities ** 2)
        return gini_impurity
        
    def calculate_gini_index(self, data, column_no, values):
        """
        Calculates the Gini index (impurity) for a specific split on a feature.

        Parameters:
            data (DataFrame): The dataset to split.
            column_no (int): The index of the feature used to split the data.
            values (list): The unique values at which to split the feature.

        Returns:
            float: The Gini index for the split.
        """
        total_samples = len(data)
        gini_index = 0.0

        for value in values:
            group_data = data[data.iloc[:, column_no] == value]  # Use iloc for column access
            group_size = len(group_data)

            if group_size == 0:  # Prevent division by zero
                continue

            proportion = group_size / total_samples
            gini_impurity = self.calculate_gini_impurity(group_data)
            gini_index += proportion * gini_impurity

        return gini_index

    def majority(self, data):
        """
        Determines the majority class label in the dataset.
        
        Parameters:
            data (DataFrame): The dataset to classify.
        
        Returns:
            int: The label of the majority class.
        """
        cls, count = np.unique(data.iloc[:, -1], return_counts=True)  # Use iloc for last column
        return cls[np.argmax(count)]

    def classify(self, data, edge):
        """
        Classifies a dataset as a leaf node.

        Parameters:
            data (DataFrame): The dataset to classify.
            edge (int): The index of the parent edge.

        Returns:
            Node: A leaf node with classification information.
        """
        classification = self.majority(data)  # Get the majority class
        gini_impurity = self.calculate_gini_impurity(data)  # Calculate Gini Impurity
        d = {
            "ID": "Leaf",
            "Classification": classification,
            "Parent_Edge": edge,
            "Gini_Impurity": gini_impurity,  # Store Gini Impurity instead of Entropy
            "Samples": data.shape[0]
        }
        leaf = Node(d)
        return leaf

    def build_tree(self, data, max_depth, attributes, edge, height):
        if data.empty:
            return None
        if height == max_depth or len(np.unique(data.iloc[:, -1])) == 1 or data.shape[0] <= 20:
            return self.classify(data, edge)

        best = {"ID": "", "best_attribute": "", "best_gini": float('inf'), "best_splits": []}

        for attribute in attributes:
            column_index = data.columns.get_loc(attribute)  # Get the column index directly
            
            splits = self.find_splits(data, column_index)
            
            # Calculate Gini index for the current attribute with all its unique values
            gini_index = self.calculate_gini_index(data, column_index, splits)
            
            if gini_index < best["best_gini"]:
                best["best_gini"] = gini_index
                best["best_attribute"] = attribute
                best["best_splits"] = splits

        if best["best_gini"] == float('inf'):
            return self.classify(data, edge)

        d = {
            "ID": best["best_attribute"],
            "Gini_Impurity": best["best_gini"],
            "Samples": data.shape[0],
            "Parent_Edge": edge,
            "Best_Splits": best["best_splits"]
        }
        node = Node(d)

        # Create child nodes for each unique value in the split
        for split_value in best["best_splits"]:
            column_index = data.columns.get_loc(best["best_attribute"])  # Get the index of the best attribute again
            child_data = data[data.iloc[:, column_index] == split_value]  # Correct boolean mask
            child_node = self.build_tree(child_data, max_depth, attributes, len(node.children), height + 1)
            node.children.append(child_node)

        return node

    def fit(self, data, max_depth=100):
        """
        Fits the decision tree model to the provided dataset using Gini Impurity.
        
        Parameters:
            data (DataFrame): The dataset to fit the tree to.
            max_depth (int): The maximum allowed depth of the tree.
        
        Returns:
            None
        """
        attributes = data.columns.tolist()[:-1]
        self.attributes = np.array(attributes)
        self.root = self.build_tree(data, max_depth, attributes, 0, 0)
        
    def plt(self, graph, node, vert):
        """
        Plots the decision tree using a graph representation.

        Parameters:
            graph (Graph): The graph object to use for plotting.
            node (Node): The current node in the tree.
            vert (int): The current vertex in the graph.

        Returns:
            str: The label of the current node.
        """
        d = node.data
        if "Classification" in node.data.keys():
            root = f'{d["ID"]}\nGini Impurity = {d["Gini_Impurity"]}\nSamples = {d["Samples"]}\nClass = {d["Classification"]}'
            graph.add_node(str(vert), label=root)
            return root
        
        root = f'{d["ID"]} in {d["Best_Splits"]}\nGini Impurity = {d["Gini_Impurity"]}\nSamples = {d["Samples"]}'
        graph.add_node(str(vert), label=root)

        for idx, child in enumerate(node.children):
            child_label = self.plt(graph, child, vert * len(node.children) + idx + 1)
            graph.add_node(str(vert * len(node.children) + idx + 1), label=child_label)
            graph.add_edge(str(vert), str(vert * len(node.children) + idx + 1))

        return root

    def predict_one(self, data):
        """
        Predicts the class label for a single data point.
        
        Parameters:
            data (np.ndarray): The data point to classify.
        
        Returns:
            int: The predicted class label.
        """
        node = self.root
        while "Classification" not in node.data.keys():  # Check if it's not a leaf node
            d = node.data
            attribute_index = np.argwhere(self.attributes == d["ID"]).squeeze()  # Get the index of the current attribute
            value = data[attribute_index]
            
            # Find the child node corresponding to the value
            found_child = False
            for child in node.children:
                # Check if child has Best_Splits and if the value matches
                if value in child.data.get("Best_Splits", []):
                    node = child
                    found_child = True
                    break
            
            # If no matching child found, break to avoid infinite loop
            if not found_child:
                break
                
        return node.data.get("Classification", None)  # Return classification or None if leaf node not found

    def predict(self, X):
        """
        Predicts the class labels for a dataset.
        
        Parameters:
            X (np.ndarray): The dataset to classify.
        
        Returns:
            np.ndarray: The predicted class labels.
        """
        predictions = []
        for x in X:
            pred = self.predict_one(x)
            if pred is None:  # Handle unclassified cases
                predictions.append(-1)  # Assign a default value, or choose a class
            else:
                predictions.append(pred)
        
        return np.array(predictions)

    def count_nodes(self, node):
        if node is None:
            return 0
        return 1 + sum(self.count_nodes(child) for child in node.children)

    def print_number_of_nodes(self):
        num_nodes = self.count_nodes(self.root)
        print(f"Number of nodes in the tree: {num_nodes}")


In [7]:
model = DecisionTree()
model.fit(train_df)

### Plotting the Decision Tree before Pruning

In [8]:
graph.draw("Decision_Tree_Before_Pruning.png", prog="dot")

In [9]:
display(Image(filename='Decision_Tree_Before_Pruning.png'))

In [10]:
model.print_number_of_nodes()

Number of nodes in the tree: 1275


### Testing 

In [11]:
X_test = test_df.values[:,:-1]
Y_test = test_df.values[:,-1]

In [12]:
Y_pred = model.predict(X_test)

In [13]:
print("Accuracy,Macro Precision, Macro Recall")
accuracy_score(Y_test,Y_pred),precision_score(Y_test,Y_pred,average='macro'),recall_score(Y_test,Y_pred,average = 'macro')

Accuracy,Macro Precision, Macro Recall


/home/jatin/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jatin/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


(0.0, 0.0, 0.0)